In [34]:
import os
from pathlib import Path
import pandas as pd

# For GCC_PHAT calc
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import rfft, irfft, fftfreq, fft, ifft
import math

from itertools import combinations
from matplotlib import animation

import subprocess
import shlex
import glob



In [35]:
# NOTE: a "frame" is the smallest unit of sampled time. (aka the data is a timeseries indexed by frame count).
# a "chunk" is a group of frames considered at a time.

# Frequency is always the same so we can hard-code it
FREQUENCY_HZ = 48000

# Always are exactly 10 seconds worth of data
# For quick testing iteration, shrink this (aka use only the first fraction of data).
N_TOTAL_FRAMES = 48000 # 480000

# Consider this-many frames of frequency data at a time.
N_FRAMES_PER_CHUNK = 480

# Chunks needn't be discrete; they can overlap. If N_ADVANCE == N_F_PER_CHUNK then they are discrete.
N_ADVANCE_PER_CHUNK = 32

ANIM_LEN_SEC = 15 # 10

# For the angle est. line in the animation
LINE_LEN = 0.15


In [36]:
# This is a publicly-available GCC-PHAT algo.

"""
 Estimate time delay using GCC-PHAT 
 Copyright (c) 2017 Yihui Xiong

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
"""

import numpy as np


def gcc_phat(sig, refsig, fs=1, max_tau=None, interp=16):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT)method.
    '''
    
    # make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    n = sig.shape[0] + refsig.shape[0]

    # Generalized Cross Correlation Phase Transform
    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = shift / float(interp * fs)
    
    return tau, cc

In [37]:
def run_anim(test_0):
    """Load a test from a given test folder, and compute the phase angles / true angles over time. Animate."""
    
    print(f"Running for test {test_0}...")
    
    pressure_data = pd.read_csv(test_0 / "pressures.csv", index_col=0, header=0)
    # display(pressure_data)
    
    positions = pd.read_csv(test_0 / "positions.csv", index_col=0, header=0)
    # display(positions)
    
    def get_pair_name(mic1_idx, mic2_idx):
        """Helper to fill/access a lookup of pairwise-computed things (delays, angles)"""
        return f"{mic1_idx}.{mic2_idx}"

    def get_chunk_data_indecies(frame_idx):
        """Convert a chunk index into indecies for the pressure data."""
        return slice(frame_idx,frame_idx+N_FRAMES_PER_CHUNK,None)

    def n_frames_to_ms(n_frames):
        """Convert from some amount of frames to time in milliseconds"""
        return n_frames/FREQUENCY_HZ*1000
    
    mic_pairs = list(combinations(range(positions.shape[0]), 2))
    
    all_pairwise_delays = []
    all_delays_flattened = []
    all_pairwise_angles = []
    all_angles_per_chunk = []
    
    # We process frequency timeseries in smaller chunks to get one estimated DOA angle per chunk of time.
    n_steps = 0
    for frame_idx in range(0, N_TOTAL_FRAMES - N_FRAMES_PER_CHUNK, N_ADVANCE_PER_CHUNK):
        n_steps += 1
        
        # Consider this chunk.
        this_chunk_pressure_data = pressure_data.iloc[get_chunk_data_indecies(frame_idx)]
        this_chunk_delays = []
        this_chunk_pairwise_delays = {}
        this_chunk_pairwise_angles = {}
        this_chunk_flattened_angles = []
        
        # Here's where we do the heavy lifting in computing phase angles for each pairing!
        for mic1_idx, mic2_idx in mic_pairs:
            mic1_pressure_header, mic2_pressure_header = this_chunk_pressure_data.columns[mic1_idx], this_chunk_pressure_data.columns[mic2_idx]

            s1, s2 = this_chunk_pressure_data[mic1_pressure_header], this_chunk_pressure_data[mic2_pressure_header]
            delay, _ = gcc_phat(s1, s2, FREQUENCY_HZ)
            
            # Geometric angle calc.
            mic1_pos = positions.iloc[mic1_idx]
            mic2_pos = positions.iloc[mic2_idx]
            m1x, m1y = tuple(mic1_pos)
            m2x, m2y = tuple(mic2_pos)
            
            # [d,a]=distang(micpos(i,:),micpos(j,:)); %Calculate distance and angle between mics
            # ra=(1:360)*pi/180-a; %Create angle vector, rotate based on mic position
            # ds=d*cos(ra)*fs/343; %Convert to sample delay
            # angularIndex(:,n)=round(ds)+bufferSize/2 +1; % Shift to put zero index in the middle


            dist, pair_edge_angle = np.sqrt((m1x-m2x)**2 + (m1y-m2y)**2), math.atan2(m2x-m1x, m2y-m1y)

            # print(delay,dist)
            speed_times_time = 343*delay/dist
            ang_relative_to_mic_edge = np.arccos(speed_times_time if np.abs(speed_times_time) <= 1 else 1 * np.sign(speed_times_time))
            geometric_angles = (ang_relative_to_mic_edge - pair_edge_angle) % (2*np.pi), (ang_relative_to_mic_edge + pair_edge_angle)  % (2*np.pi)

            pair_idx = get_pair_name(mic1_idx, mic2_idx)
            
            this_chunk_delays.append(delay)
            this_chunk_pairwise_delays[pair_idx] = delay
            this_chunk_pairwise_angles[pair_idx] = geometric_angles
            this_chunk_flattened_angles += list(geometric_angles)
                    
        all_pairwise_delays.append(this_chunk_pairwise_delays)
        all_delays_flattened += this_chunk_delays
        all_pairwise_angles.append(this_chunk_pairwise_angles)
        all_angles_per_chunk.append(this_chunk_flattened_angles)
     
    
     
    # Create an animation 
    fig, ax = plt.subplots()
    ax.set_title(f"{test_0}")
    t_text = ax.text(0.007, 0.03, "t")
        
    # Setup the orignal line color and positions of the animation.
    # Use the last chunk done (bound to "this_chunk_pressure_data") to grab headers.
    lines = []

    for mic1_idx, mic2_idx in mic_pairs:
        mic1_pos = positions.iloc[mic1_idx]
        mic2_pos = positions.iloc[mic2_idx]
        m1x, m1y = tuple(mic1_pos)
        m2x, m2y = tuple(mic2_pos)
        
        angle, symmetric_angle = this_chunk_pairwise_angles[get_pair_name(mic1_idx, mic2_idx)]

        
        # move edges right by factor of line length to remove overlaps
        xo = np.sqrt((m1x - m2x)**2 + (m1y-m2y)**2) * 0.03
        
        # Graph the mic edge
        mic_edge_line, = ax.plot([m1x+xo, m2x+xo], [m1y, m2y], c=(1, 0, 0))
        
        ANGLE_LINE_ALPHA = 0.1
        ANGLE_LINE_LW = 3
        
        # Graph possible angle and its symmetry
        angle_line, = ax.plot([0, LINE_LEN*np.cos(angle)], [0, LINE_LEN*np.sin(angle)], c=(0, 1, 0), lw=ANGLE_LINE_LW, alpha=ANGLE_LINE_ALPHA)
        angle_symmetric_line, = ax.plot([0, -LINE_LEN*np.cos(symmetric_angle)], [0, -LINE_LEN*np.sin(symmetric_angle)], c=(0, 1, 0), lw=ANGLE_LINE_LW, alpha=ANGLE_LINE_ALPHA)
        lines.append((mic_edge_line, angle_line, angle_symmetric_line))
  
    max_delay, min_delay = max(all_delays_flattened), min(all_delays_flattened)
 
    def update(chunk_idx):
        """For our animation. For each chunk create one animation frame, in which we update the line colors."""

        frame_idx = chunk_idx*N_ADVANCE_PER_CHUNK

        this_chunk_pressure_data = pressure_data[get_chunk_data_indecies(frame_idx)]
        this_chunk_pairwise_delays = all_pairwise_delays[chunk_idx]
        this_chunk_pairwise_angles = all_pairwise_angles[chunk_idx]
        this_chunk_all_angles = all_angles_per_chunk[chunk_idx]
        
        
        # For each frame, we need to recolor each edge based on the delays of this chunk.
        for lines_idx, (mic1_idx, mic2_idx) in enumerate(mic_pairs):
            pair_idx = get_pair_name(mic1_idx, mic2_idx)
            delay = this_chunk_pairwise_delays[pair_idx]
            
            # Since all time delays are relatively close to eachother,
            # scale the color by first normalizing then putting through an exponential.
            # Makes higher values *much* more intense.
            EXAGGERATION = 2.
            alpha = 1 if max_delay == min_delay else np.exp(EXAGGERATION*(delay - min_delay) / (max_delay - min_delay)) / (np.exp(EXAGGERATION))

            color = (1, 0, 0)
 
            mic_edge_line, angle_line, angle_symmetric_line = lines[lines_idx]
 
            mic_edge_line.set_alpha(alpha*0.6)
            mic_edge_line.set_color(color)
            
            angle, symmetric_angle = this_chunk_pairwise_angles[pair_idx]
            angle_line.set_xdata([0, LINE_LEN*np.cos(angle)])
            angle_line.set_ydata([0, LINE_LEN*np.sin(angle)])
            angle_symmetric_line.set_xdata([0, LINE_LEN*np.cos(symmetric_angle)])
            angle_symmetric_line.set_ydata([0, LINE_LEN*np.sin(symmetric_angle)])
            
        t_text.set_text((
            f"Step # {chunk_idx}\n"
            f"t={n_frames_to_ms(frame_idx):.2f}ms\n"
            f"Frame # {frame_idx}\n"
            f"Chunk size:\n  {N_FRAMES_PER_CHUNK} frames ({n_frames_to_ms(N_FRAMES_PER_CHUNK):.2f}ms)\n"
            f"frame adv. per chunk:\n  {N_ADVANCE_PER_CHUNK} frames ({n_frames_to_ms(N_ADVANCE_PER_CHUNK):.2f}ms)\n"
            "Darker=longer delay"
        ))
            
        return *lines, t_text,
    
    # Save the animation.
    ani = animation.FuncAnimation(fig, update, frames=n_steps, interval=ANIM_LEN_SEC * 1000 / n_steps)
    test_0_same_dir= test_0.relative_to(*test_0.parts[:1])
    ani.save(f"{test_0_same_dir}_anim.mp4")
    plt.gcf().set_visible(False)
    


In [38]:
def run_all_test_anims():
    """Run and animate each timeseries test, with angles computed. Outputs saved alongside this file."""
    test_dirs = [f for f in Path("Data").iterdir() if f.is_dir()]

    for test in test_dirs:  
        run_anim(test)

In [39]:
run_all_test_anims()

Running for test Data/test_01_white_noise_0_fwd...


/tmp/ipykernel_15524/1410709382.py:37: RuntimeWarning: invalid value encountered in divide
  cc = np.fft.irfft(R / np.abs(R), n=(interp * n))


KeyboardInterrupt: 

(Animations are alongside this file. Not embedded yet because refreshing them every time is a chore.)

In [47]:
def combine_test_videos():
    """Create a unified video that combines the test videos."""
    files = []
    for f in glob.glob("test_*.mp4", root_dir="."):
        files.append(f)
    print(files)
    if len(files) >= 5:
        subprocess.run(shlex.split(f"""ffmpeg -y \
    -i {files[0]} -i {files[1]} \
    -i {files[2]} -i {files[3]} \
    -i {files[4]} -i {files[4]} \
    -filter_complex \
    "[0:v][1:v][2:v]hstack=inputs=3[top];\
    [3:v][4:v][5:v]hstack=inputs=3[bottom];\
    [top][bottom]vstack=inputs=2[v]" \
    -map "[v]" \
    all_tests_anim.mp4"""))


In [48]:
combine_test_videos()

['test_01_white_noise_0_fwd_anim.mp4', 'test_02_white_noise_45_left_anim.mp4', 'test_03_white_noise_90_left_anim.mp4', 'test_04_engine_noise_no_talking_anim.mp4', 'test_06_engine_noise_talking_anim.mp4']


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e